In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
import numpyro.distributions as dist
from hbmep.model import Baseline


class Logistic5(Baseline):
    LINK = "Logistic5"

    def __init__(self, config: Config):
        super(Logistic5, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        feature1 = features[1].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]
        n_feature1 = np.unique(feature1).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            global_sigma_b = numpyro.sample("global_sigma_b", dist.HalfNormal(5))
            global_sigma_v = numpyro.sample("global_sigma_v", dist.HalfNormal(5))

            global_sigma_L = numpyro.sample("global_sigma_L", dist.HalfNormal(1))
            global_sigma_H = numpyro.sample("global_sigma_H", dist.HalfNormal(5))

            global_sigma_g_1 = numpyro.sample("global_sigma_g_1", dist.HalfNormal(100))
            global_sigma_g_2 = numpyro.sample("global_sigma_g_2", dist.HalfNormal(100))

            global_sigma_p = numpyro.sample("global_sigma_p", dist.HalfNormal(5))

            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                with numpyro.plate("n_feature1", n_feature1, dim=-3):
                    """ Hyper-priors """
                    mu_a = numpyro.sample(site.mu_a, dist.HalfNormal(scale=10))
                    sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(scale=1))

                    sigma_b_raw = numpyro.sample("sigma_b_raw", dist.HalfNormal(scale=1))
                    sigma_b = numpyro.deterministic(site.sigma_b, global_sigma_b * sigma_b_raw)

                    sigma_v_raw = numpyro.sample("sigma_v_raw", dist.HalfNormal(scale=1))
                    sigma_v = numpyro.deterministic(site.sigma_v, global_sigma_v * sigma_v_raw)

                    sigma_L_raw = numpyro.sample("sigma_L_raw", dist.HalfNormal(scale=1))
                    sigma_L = numpyro.deterministic(site.sigma_L, global_sigma_L * sigma_L_raw)

                    sigma_H_raw = numpyro.sample("sigma_H_raw", dist.HalfNormal(scale=1))
                    sigma_H = numpyro.deterministic(site.sigma_H, global_sigma_H * sigma_H_raw)

                    sigma_g_1_raw = numpyro.sample("sigma_g_1_raw", dist.HalfNormal(scale=1))
                    sigma_g_1 = numpyro.deterministic("sigma_g_1", global_sigma_g_1 * sigma_g_1_raw)

                    sigma_g_2_raw = numpyro.sample("sigma_g_2_raw", dist.HalfNormal(scale=1))
                    sigma_g_2 = numpyro.deterministic("sigma_g_2", global_sigma_g_2 * sigma_g_2_raw)

                    sigma_p_raw = numpyro.sample("sigma_p_raw", dist.HalfNormal(scale=1))
                    sigma_p = numpyro.deterministic("sigma_p", global_sigma_p * sigma_p_raw)

                    with numpyro.plate(site.n_subject, n_subject, dim=-4):
                        """ Priors """
                        a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=mu_a, rate=1))
                        a = numpyro.deterministic(site.a, (1 / sigma_a) * a_raw)

                        b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                        b = numpyro.deterministic(site.b, sigma_b * b_raw)

                        v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                        v = numpyro.deterministic(site.v, sigma_v * v_raw)

                        L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                        L = numpyro.deterministic(site.L, sigma_L * L_raw)

                        H_raw = numpyro.sample("H_raw", dist.HalfNormal(scale=1))
                        H = numpyro.deterministic(site.H, sigma_H * H_raw)

                        g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                        g_1 = numpyro.deterministic(site.g_1, sigma_g_1 * g_1_raw)

                        g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                        g_2 = numpyro.deterministic(site.g_2, sigma_g_2 * g_2_raw)

                        p_raw = numpyro.sample("p_raw", dist.HalfNormal(scale=1))
                        p = numpyro.deterministic("p", sigma_p * p_raw)

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[subject, feature1, feature0]
            + (
                H[subject, feature1, feature0]
                / jnp.power(
                    1 + v[subject, feature1, feature0] * jnp.exp(-b[subject, feature1, feature0] * (intensity - a[subject, feature1, feature0])),
                    1 / v[subject, feature1, feature0]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[subject, feature1, feature0] + g_2[subject, feature1, feature0] * ((1 / (mu + 1)) ** p[subject, feature1, feature0])
        )

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


In [3]:
root_path = Path(os.getcwd()).parent.parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/paper/rats/SHIE/Logistic5.toml")

config = Config(toml_path=toml_path)
config.MCMC_PARAMS["num_warmup"] = 4000
config.MCMC_PARAMS["num_samples"] = 6000
config.MCMC_PARAMS["thinning"] = 4

model = Logistic5(config=config)


2023-10-13 13:36:31,490 - hbmep.config - INFO - Verifying configuration ...
2023-10-13 13:36:31,491 - hbmep.config - INFO - Success!
2023-10-13 13:36:31,505 - hbmep.model.baseline - INFO - Initialized model with Logistic5 link


In [4]:
src = "/home/vishu/data/hbmep-processed/L_SHIE/data.csv"
df = pd.read_csv(src)

model.mep_matrix_path = None
# subset = ["amap01", "amap02"]
# ind = df[model.subject].isin(subset)
# df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-10-13 13:36:31,593 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/rats/SHIE/Logistic5
2023-10-13 13:36:31,594 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/rats/SHIE/Logistic5
2023-10-13 13:36:31,595 - hbmep.dataset.core - INFO - Processing data ...
2023-10-13 13:36:31,597 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-10-13 13:36:31,672 - hbmep.model.baseline - INFO - Running inference with Logistic5 ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-10-13 15:13:05,421 - hbmep.utils.utils - INFO - func:run_inference took: 1 hr and 36 min


In [11]:
mcmc.print_summary(prob=.95)



                          mean       std    median      2.5%     97.5%     n_eff     r_hat
      H_raw[0,0,0,0]      0.70      0.16      0.66      0.45      1.01      2.85      2.38
      H_raw[0,0,0,1]      0.53      0.39      0.41      0.10      1.28     23.66      1.22
      H_raw[0,0,0,2]      0.23      0.04      0.23      0.16      0.32     21.51      1.17
      H_raw[0,0,0,3]      0.09      0.02      0.09      0.05      0.11      3.25      1.87
      H_raw[0,0,0,4]      0.24      0.04      0.23      0.17      0.32      7.71      1.32
      H_raw[0,0,0,5]      0.57      0.16      0.53      0.37      0.80     14.99      1.23
      H_raw[0,0,1,0]      1.11      0.43      1.09      0.34      1.91     33.21      1.13
      H_raw[0,0,1,1]      0.12      0.07      0.10      0.04      0.29      3.38      2.02
      H_raw[0,0,1,2]      0.19      0.30      0.06      0.01      0.87      4.33      1.55
      H_raw[0,0,1,3]      0.18      0.13      0.15      0.04      0.36     19.13      1.2

In [7]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-10-13 15:13:07,412 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec


2023-10-13 15:14:30,319 - hbmep.utils.utils - INFO - func:predict took: 1 min and 22.91 sec
2023-10-13 15:14:30,551 - hbmep.model.baseline - INFO - Rendering ...
2023-10-13 15:16:16,131 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/rats/SHIE/Logistic5/recruitment_curves.pdf
2023-10-13 15:16:16,134 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 1 min and 45.81 sec
2023-10-13 15:16:16,592 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-10-13 15:18:18,344 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/rats/SHIE/Logistic5/posterior_predictive_check.pdf
2023-10-13 15:18:18,350 - hbmep.utils.utils - INFO - func:_render_predictive_check took: 2 min and 2.22 sec
2023-10-13 15:18:18,350 - hbmep.utils.utils - INFO - func:render_predictive_check took: 2 min and 2.22 sec


In [8]:
numpyro_data = az.from_numpyro(mcmc)

""" Model evaluation """
logger.info("Evaluating model ...")


2023-10-13 15:18:33,460 - __main__ - INFO - Evaluating model ...


In [9]:
score = az.loo(numpyro_data)
logger.info(f"ELPD LOO (Log): {score.elpd_loo:.2f}")

/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-10-13 15:19:46,036 - __main__ - INFO - ELPD LOO (Log): 36411.90


In [10]:
score = az.waic(numpyro_data)
logger.info(f"ELPD WAIC (Log): {score.elpd_waic:.2f}")

/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
2023-10-13 15:19:46,369 - __main__ - INFO - ELPD WAIC (Log): 36334.18


In [12]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)


In [12]:
# import pickle

# with open(dest, "rb") as g:
#     model_, mcmc_, posterior_samples_ = pickle.load(g)
